In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load your specific input files
img1 = cv2.imread("image_panaroma/input1.png")
img2 = cv2.imread("image_panaroma/input2.png")

# Store in a list
imgs = [img1, img2]

# Check if images loaded
for i, img in enumerate(imgs):
    if img is None:
        print(f"Image failed to load")
    else:
        print(f"Image loaded")


Image loaded
Image loaded


In [16]:
sift = cv2.SIFT_create()
keypoints = []
descriptors = []

for img in imgs:
    kp, des = sift.detectAndCompute(img, None)
    keypoints.append(kp)
    descriptors.append(des)


In [18]:
from sklearn.metrics.pairwise import cosine_similarity

num_imgs = len(descriptors)
adjacency_matrix = np.zeros((num_imgs, num_imgs), dtype=int)
np.fill_diagonal(adjacency_matrix, 1)

for idx1 in range(num_imgs):
    for idx2 in range(idx1 + 1, num_imgs):
        desc_a = descriptors[idx1]
        desc_b = descriptors[idx2]

        if desc_a is None or desc_b is None:
            continue
            
        avg_sim = np.mean(cosine_similarity(desc_a, desc_b))
        
        matches_a_to_b = []
        matches_b_to_a = []
        
        for i, vec_a in enumerate(desc_a):
            distances = np.linalg.norm(desc_b - vec_a, axis=1)
            best_match = np.argmin(distances)
            matches_a_to_b.append((i, best_match))
            
        for j, vec_b in enumerate(desc_b):
            distances = np.linalg.norm(desc_a - vec_b, axis=1)
            best_match = np.argmin(distances)
            matches_b_to_a.append((best_match, j))
            
        mutual_matches = set(matches_a_to_b) & set(matches_b_to_a)
        
        good_matches = []
        for i, vec_a in enumerate(desc_a):
            distances = np.linalg.norm(desc_b - vec_a, axis=1)
            sorted_indices = np.argsort(distances)
            if distances[sorted_indices[0]] < 0.7 * distances[sorted_indices[1]]:
                good_matches.append((i, sorted_indices[0]))
                
        min_keypoints = min(len(desc_a), len(desc_b))
        threshold = max(4, 0.15 * min_keypoints)
        
        if (len(mutual_matches) > threshold) or (avg_sim > 0.5 and len(good_matches) > threshold//2):
            adjacency_matrix[idx1][idx2] = 1
            adjacency_matrix[idx2][idx1] = 1

In [20]:
connection_counts = np.sum(adjacency_matrix, axis=1)
reference_idx = np.argmax(connection_counts)

num_images = len(descriptors)
homographies = [np.eye(3) if i == reference_idx else None for i in range(num_images)]

match_ratio_threshold = 0.7
min_match_ratio = 0.15
ransac_thresh = 3.0

for current_idx in range(num_images):
    if current_idx == reference_idx:
        continue
    if descriptors[current_idx] is None or descriptors[reference_idx] is None:
        continue
    
    current_desc = descriptors[current_idx]
    reference_desc = descriptors[reference_idx]
    good_matches = []
    
    for feat_idx, feat_vec in enumerate(current_desc):
        distances = np.linalg.norm(reference_desc - feat_vec, axis=1)
        sorted_indices = np.argsort(distances)
        best_dist = distances[sorted_indices[0]]
        second_dist = distances[sorted_indices[1]]
        
        if best_dist < match_ratio_threshold * second_dist:
            good_matches.append((feat_idx, sorted_indices[0]))
    
    min_kpts = min(len(keypoints[current_idx]), len(keypoints[reference_idx]))
    required_matches = max(4, int(min_kpts * min_match_ratio))
    
    if len(good_matches) >= required_matches:
        src_points = np.float32([keypoints[current_idx][m[0]].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_points = np.float32([keypoints[reference_idx][m[1]].pt for m in good_matches]).reshape(-1, 1, 2)
        
        H, _ = cv2.findHomography(src_points, dst_points, cv2.RANSAC, ransac_thresh)
        
        if H is not None:
            homographies[current_idx] = H

In [22]:
h_ref, w_ref = imgs[reference_idx].shape[:2]
corners = np.float32([[0, 0], [0, h_ref], [w_ref, h_ref], [w_ref, 0]]).reshape(-1, 1, 2)

all_corners = []
for i in range(num_imgs):
    if homographies[i] is None:
        continue
    h_i, w_i = imgs[i].shape[:2]
    img_corners = np.float32([[0, 0], [0, h_i], [w_i, h_i], [w_i, 0]]).reshape(-1, 1, 2)
    warped_corners = cv2.perspectiveTransform(img_corners, homographies[i])
    all_corners.append(warped_corners)

all_pts = np.concatenate(all_corners, axis=0)
[xmin, ymin] = np.int32(all_pts.min(axis=0).ravel() - 0.5)
[xmax, ymax] = np.int32(all_pts.max(axis=0).ravel() + 0.5)

width = xmax - xmin
height = ymax - ymin
translation = np.array([[1, 0, -xmin], [0, 1, -ymin], [0, 0, 1]])

In [24]:
panorama = np.zeros((height, width, 3), dtype=np.uint8)

for i in range(num_imgs):
    if homographies[i] is None:
        continue
    H_translated = translation @ homographies[i]
    warped_img = cv2.warpPerspective(imgs[i], H_translated, (width, height))
    mask = (panorama == 0)
    panorama[mask] = warped_img[mask]

cv2.imshow("Panorama", panorama)
cv2.imwrite("output.png", panorama)

True